# Fine-tune BERT as Benchmark

- Fine tune BERT model using autotrain from Huggingface: https://huggingface.co/autotrain
- Used the train.csv and val.csv in the autotrain job
- Use default configuration
- Fine-tuned model : https://huggingface.co/vincentclaes/autotrain-0br8k-gdjpm
- accuracy: 0.9994418604651163

## Run against test dataset and calculate the accuracy

In [1]:
!pip install transformers scikit-learn --quiet

In [2]:
from transformers import pipeline

# Load the model (if you're using the model from the Hugging Face Hub)
classifier = pipeline('text-classification', model='vincentclaes/autotrain-0br8k-gdjpm')

# Inference
result = classifier("Your input text here")
print(result)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'ACCOUNT', 'score': 0.9998332262039185}]


In [3]:
# read test.csv, and make prediction for the column 'question' and add the result to the column 'predicted_class_name'
import pandas as pd
df = pd.read_csv('test.csv')
df['predicted_class_name'] = df['question'].apply(lambda x: classifier(x)[0]['label'])
df

,class_name,question,id,predicted_class_name
0,CANCEL,i do not know how i can see the cancellation c...,3329,CANCEL
1,SUBSCRIPTION,help me cancelling my corporate newsletter sub...,17714,SUBSCRIPTION
2,REFUND,I paid {{Currency Symbol}}{{Refund Amount}} fo...,16152,REFUND
3,CANCEL,i want assistance checking the termination pen...,3648,CANCEL
4,REFUND,i expect a rebate of {{Currency Symbol}}{{Refu...,26431,REFUND
...,...,...,...,...
325,CONTACT,"I want to speak with customer service, helpme",8258,CONTACT
326,INVOICE,get invoice #85632,15244,INVOICE
327,FEEDBACK,help me filing a complaint,7352,FEEDBACK
328,DELIVERY,want help seeing how long it takes for my ship...,13018,DELIVERY


In [4]:
# use the huggingface evaluate library to evaluate the model by taking the columns
# 'predicted_class_name' and 'class_name' as input andd calculate the accuracy
from datasets import load_metric
# Create a mapping of class names to numerical labels
unique_classes = set(df['predicted_class_name']).union(set(df['class_name']))
class_to_int = {cls_name: idx for idx, cls_name in enumerate(unique_classes)}

# Map the class names to integers
df['predicted_class_numeric'] = df['predicted_class_name'].map(class_to_int)
df['class_numeric'] = df['class_name'].map(class_to_int)

# Compute the accuracy
metric = load_metric("accuracy")
accuracy = metric.compute(predictions=df['predicted_class_numeric'], references=df['class_numeric'])

print(f"Accuracy: {accuracy['accuracy']}")


/var/folders/sj/3t0z94yj4m12rfbjsb8x5j5r0000gn/T/ipykernel_5045/2505427709.py:13: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


Accuracy: 0.996969696969697


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
